In [1]:
from sqlalchemy import create_engine
import sqlite3
import pandas as pd

In [2]:
try:
    sqliteConnection = sqlite3.connect('./SQLFiles Tier 2/sqlite_db_pythonsqlite.db') # create a new database if it cant find a local one
    cursor = sqliteConnection.cursor() # create a cursor object to execute SQL commands from Python
    print("Database created and Successfully Connected to SQLite")

    sqlite_select_Query = "select sqlite_version();"
    cursor.execute(sqlite_select_Query) # use execute to run SQL commands
    record = cursor.fetchall() #read results
    print("SQLite Database Version is: ", record)
    #cursor.close()

except sqlite3.Error as error:
    print("Error while connecting to sqlite", error)
#finally:
#    if sqliteConnection:
#        sqliteConnection.close()
#        print("The SQLite connection is closed")

Database created and Successfully Connected to SQLite
SQLite Database Version is:  [('3.35.4',)]


Q.10 Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [4]:
rs1 = cursor.execute("SELECT name, SUM(revenue) FROM (SELECT f.facid, f.name, CASE WHEN b.memid = 0 THEN guestcost*slots ELSE membercost*slots END AS revenue FROM Facilities AS f LEFT JOIN Bookings AS b ON b.facid = f.facid) AS sub_query GROUP BY facid ORDER BY revenue")
results1 = pd.DataFrame(rs1.fetchall())
results1.columns = ["Facility_name", "revenue"]
print(results1)

     Facility_name  revenue
0       Pool Table    270.0
1     Table Tennis    180.0
2    Snooker Table    240.0
3     Squash Court  13468.0
4  Badminton Court   1906.5
5   Tennis Court 1  13860.0
6   Tennis Court 2  14310.0
7   Massage Room 1  50351.6
8   Massage Room 2  14454.6


Q.11 Produce a report of members and who recommended them in alphabetic surname,firstname order

In [5]:
rs2 = cursor.execute("SELECT DISTINCT surname|| ' ' || firstname AS members, sub_query.recommended_by FROM Members LEFT JOIN (SELECT m1.recommendedby, CASE WHEN m2.memid >0 THEN m2.surname || ' ' || m2.firstname ELSE '' END AS recommended_by FROM Members AS m1 LEFT JOIN Members AS m2 ON m1.recommendedby = m2.memid) AS sub_query ON Members.recommendedby = sub_query.recommendedby WHERE Members.recommendedby >0 ORDER BY surname")
results2 = pd.DataFrame(rs2.fetchall())
results2.columns = ["member", "recommendedby"]
print(results2)

                     member      recommendedby
0            Bader Florence    Stibbons Ponder
1                Baker Anne    Stibbons Ponder
2             Baker Timothy     Farrell Jemima
3                Boothe Tim         Rownam Tim
4            Butters Gerald       Smith Darren
5               Coplin Joan      Baker Timothy
6             Crumpet Erica        Smith Tracy
7                Dare Nancy    Joplette Janice
8           Genting Matthew     Butters Gerald
9                 Hunt John  Purview Millicent
10              Jones David    Joplette Janice
11            Jones Douglas        Jones David
12          Joplette Janice       Smith Darren
13           Mackenzie Anna       Smith Darren
14             Owen Charles       Smith Darren
15             Pinker David     Farrell Jemima
16        Purview Millicent        Smith Tracy
17         Rumney Henrietta    Genting Matthew
18         Sarwin Ramnaresh     Bader Florence
19               Smith Jack       Smith Darren
20          S

Q.12 Find the facilities with their usage by member, but not guests

In [6]:
rs3 = cursor.execute("SELECT name, SUM(slots) AS member_usage_slots FROM Facilities as f LEFT JOIN Bookings as b ON f.facid = b.facid WHERE b.memid > 0 GROUP BY f.facid")
results3 = pd.DataFrame(rs3.fetchall())
results3.columns = ["name", "member usage (slots)"]
print(results3)

              name  member usage (slots)
0   Tennis Court 1                   957
1   Tennis Court 2                   882
2  Badminton Court                  1086
3     Table Tennis                   794
4   Massage Room 1                   884
5   Massage Room 2                    54
6     Squash Court                   418
7    Snooker Table                   860
8       Pool Table                   856


Q.13 Find the facilities usage by month, but not guests

In [8]:
rs4 = cursor.execute("SELECT name, strftime('%m', starttime) AS month, SUM(slots) AS member_usage_slots FROM Facilities as f LEFT JOIN Bookings as b ON f.facid = b.facid WHERE b.memid > 0 GROUP BY f.facid, month ORDER BY month, member_usage_slots")
results4 = pd.DataFrame(rs4.fetchall())
results4.columns = ["name","month", "member usage (slots)"]
print(results4)

               name month  member usage (slots)
0    Massage Room 2    07                     8
1      Squash Court    07                    50
2      Table Tennis    07                    98
3        Pool Table    07                   110
4    Tennis Court 2    07                   123
5     Snooker Table    07                   140
6   Badminton Court    07                   165
7    Massage Room 1    07                   166
8    Tennis Court 1    07                   201
9    Massage Room 2    08                    18
10     Squash Court    08                   184
11     Table Tennis    08                   296
12       Pool Table    08                   303
13   Massage Room 1    08                   316
14    Snooker Table    08                   316
15   Tennis Court 1    08                   339
16   Tennis Court 2    08                   345
17  Badminton Court    08                   414
18   Massage Room 2    09                    28
19     Squash Court    09               